In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import import_ipynb
from util import get_parking_spots_bboxes, empty_or_not


In [4]:
def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))


mask = './mask_1920_1080.png'
video_path = './samples/parking_1920_1080_loop.mp4'


mask = cv2.imread(mask, 0)

cap = cv2.VideoCapture(video_path)

connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

spots = get_parking_spots_bboxes(connected_components)

spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None



In [ ]:
frame_nmr = 0
ret = True
step = 30
while ret:
    ret, frame = cap.read()

    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

        print([diffs[j] for j in np.argsort(diffs)][::-1])

    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            spot_status = empty_or_not(spot_crop)

            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1
    
cap.release()
cv2.destroyAllWindows()

[7.617822128851543, 7.263607085346209, 7.152090621355995, 7.012660336498428, 6.423148148148144, 6.359730848861275, 6.05290736984449, 6.042973856209144, 6.013659836748303, 5.896594982078852, 5.873680443728745, 5.7584541062801975, 5.753456605030806, 5.623039215686276, 5.601190476190482, 5.5905797101449295, 5.587394957983193, 5.582375478927204, 5.579188712522054, 5.57302231237324, 5.549619556418975, 5.461102781942358, 5.379810094952518, 5.316747181964573, 5.303657694962048, 5.289694041867946, 5.1927369648509085, 5.190500338066258, 5.178291062801932, 5.17728758169936, 5.168464052287575, 5.144827586206901, 5.105476673427987, 5.064437478230573, 5.057165861513695, 5.04589371980677, 5.0421900161030635, 5.037072701011084, 5.006901311249138, 4.958412397216961, 4.8996848739495675, 4.897551978537905, 4.871703853955381, 4.8256302521008365, 4.803571428571431, 4.784482758620669, 4.720772946859896, 4.690154922538724, 4.684955141476884, 4.617816091954026, 4.607524520567978, 4.603594771241831, 4.5755868